# Contents of This Program
- Demonstration of auto-calibartion -- just run this notebook, after changing setttings in ./program/04_calib_outsourced.py

In [4]:
PROGRAM_NAME = "06_repeat_single_shot_v2"

In [5]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [ ]:
timeout = 90 * 60
elapsed_time = 0.0
payloads_calib = []
payloads_res = []

import time

count = 0

while True:
    print(f"=== iteration: {count} ===")

    start_time = time.time()

    payloads = experiment(PROGRAM_NAME)

    end_time = time.time()
    elapsed = end_time - start_time
    elapsed_time += elapsed
    print(f"elapsed_time = {elapsed_time} seconds")

    print(f"len(payloads) = {len(payloads)}")
    print(payloads[0])

    payloads_calib.append(payloads[0])
    payloads_res.extend(payloads[1:])

    if elapsed_time > timeout:
        print("Timeout reached. Stopping the loop.")
        break

    count += 1

=== iteration: 0 ===
job_id = 069553d4-1ea9-7376-8000-c3fdf2c6c9c6
